In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'final_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,11.0,15.0,26.0,8.0,9.0,7.0,3.0,1.0,0.0,3.0,...,11.0,7.0,4.0,3.0,4.0,9.0,19.0,2022-01-29,4,17.0
1,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,5.0,4.0,10.0,7.0,5.0,9.0,10.0,2022-01-30,4,9.0
2,0.0,1.0,0.0,0.0,1.0,1.0,1.0,3.0,2.0,3.0,...,8.0,7.0,8.0,5.0,5.0,10.0,0.0,2022-01-31,4,3.0
3,1.0,1.0,0.0,0.0,0.0,3.0,2.0,3.0,4.0,5.0,...,3.0,16.0,7.0,1.0,0.0,1.0,3.0,2022-02-01,4,3.0
4,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,2.0,...,3.0,8.0,3.0,0.0,4.0,4.0,3.0,2022-02-02,4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-27,199,0.0
88290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-28,199,0.0
88291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-29,199,0.0
88292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-30,199,0.0


In [3]:
from src.data_split import train_test_split
from datetime import datetime

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 8, 1, 0, 0, 0),
    target_column_name="target_rides_next_hour"
)

In [4]:
print(f"{X_train.shape=}")
print(f"{y_train.shape=}")
print(f"{X_test.shape=}")
print(f"{y_test.shape=}")

X_train.shape=(48208, 674)
y_train.shape=(48208,)
X_test.shape=(40086, 674)
y_test.shape=(40086,)


In [5]:
from src.model import get_pipeline
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
import numpy as np

import optuna
def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average validation error based on a TimeSeriesSplit
    """

    # pick hyper-parameters
    hyperparams = {
        "metric": "mae",
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),
    }

    tss = TimeSeriesSplit(n_splits=4)
    scores = []
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]

        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)

        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)

    
    # return the mean score
    return np.array(scores).mean()

In [6]:
study = optuna.create_study(direction="minimize") # minimize the mean mae score
study.optimize(objective, n_trials=5)

[I 2025-02-03 12:18:48,845] A new study created in memory with name: no-name-fbe5363f-0442-4808-9545-db21cd6d868f
[I 2025-02-03 12:19:01,409] Trial 0 finished with value: 1.554964788738914 and parameters: {'num_leaves': 47, 'feature_fraction': 0.857745326657342, 'bagging_fraction': 0.2047226593775144, 'min_child_samples': 57}. Best is trial 0 with value: 1.554964788738914.
[I 2025-02-03 12:19:24,399] Trial 1 finished with value: 1.456579315154382 and parameters: {'num_leaves': 137, 'feature_fraction': 0.4224814398821883, 'bagging_fraction': 0.7295927390439612, 'min_child_samples': 64}. Best is trial 1 with value: 1.456579315154382.
[I 2025-02-03 12:19:30,428] Trial 2 finished with value: 1.7713886212909447 and parameters: {'num_leaves': 8, 'feature_fraction': 0.9827062987420179, 'bagging_fraction': 0.3545296866584014, 'min_child_samples': 79}. Best is trial 1 with value: 1.456579315154382.
[I 2025-02-03 12:19:55,999] Trial 3 finished with value: 1.4467466916679488 and parameters: {'num

In [7]:
best_params = study.best_trial.params
print(f"{best_params=}")

best_params={'num_leaves': 149, 'feature_fraction': 0.4267530900437342, 'bagging_fraction': 0.4230262377528063, 'min_child_samples': 61}


#### Once we find the best parameter, re train the model with entire train data from the initial dataset

In [8]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function avg_rides_last_4_weeks at 0x12f9e8040>)),
                ('temporalfeatureengineering', TemporalFeatureEngineering()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.4230262377528063,
                               feature_fraction=0.4267530900437342,
                               min_child_samples=61, num_leaves=149))])

In [9]:
predictions = pipeline.predict(X_test)
predictions

array([3.14736681, 2.65299032, 2.50950271, ..., 0.02296284, 0.02296284,
       0.02512078])

In [10]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae=:.4f}")

test_mae=2.5627


In [11]:
from src.plot import plot_one_sample

plot_one_sample(
    features=X_test,
    targets=y_test,
    example_id=2977,
    predictions=pd.Series(predictions)
)

In [12]:
plot_one_sample(
    features=X_test,
    targets=y_test,
    example_id=100,
    predictions=pd.Series(predictions)
)

In [13]:
plot_one_sample(
    features=X_test,
    targets=y_test,
    example_id=32085,
    predictions=pd.Series(predictions)
)